In [1]:
from tqdm import tqdm as tqdm
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import DataStructs
from Bio.SeqUtils.ProtParam import ProteinAnalysis
import numpy as np
import pandas as pd
from sklearn.preprocessing import normalize
from itertools import product
import math

def get_fingerprint(smiles, r=3, nBits=1024):
    compound = Chem.MolFromSmiles(smiles.strip())
    fingerprint = AllChem.GetMorganFingerprintAsBitVect(compound, r, nBits=nBits)
    m = np.zeros((0, ), dtype=np.int8)
    DataStructs.ConvertToNumpyArray(fingerprint,m)
    return m

def sequence_to_kmer(protein_seq, k):
    groups={'A':'1','V':'1','G':'1','I':'2','L':'2','F':'2','P':'2','Y':'3',
            'M':'3','T':'3','S':'3','H':'4','N':'4','Q':'4','W':'4',
            'R':'5','K':'5','D':'6','E':'6','C':'7'}
    crossproduct=[''.join (i) for i in product("1234567",repeat=k)]
    for i in range(0, len(crossproduct)): crossproduct[i]=int(crossproduct[i])
    ind=[]
    for i in range(0, len(crossproduct)): ind.append(i)
    combinations=dict(zip(crossproduct, ind))
    
    V=np.zeros(int((math.pow(7,k))))
    try:
        for j in range(0, len(protein_seq)-k+1):
            kmer=protein_seq[j:j+k]
            c=''
            for l in range(0, k):
                c+=groups[kmer[l]]
                V[combinations[int(c)]] += 1
    except:
        count={'1':0,'2':0,'3':0, '4':0, '5':0,'6':0,'7':0}
        for q in range(0,len(protein_seq)):
            if protein_seq[q]=='A' or protein_seq[q]=='V' or protein_seq[q]=='G':
                count['1']+=1
            if protein_seq[q]=='I' or protein_seq[q]=='L'or protein_seq[q]=='F' or protein_seq[q]=='P':
                count['2']+=1
            if protein_seq[q]=='Y' or protein_seq[q]=='M'or protein_seq[q]=='T' or protein_seq[q]=='S':
                count['3']+=1
            if protein_seq[q]=='H' or protein_seq[q]=='N'or protein_seq[q]=='Q' or protein_seq[q]=='W':
                count['4']+=1
            if protein_seq[q]=='R' or protein_seq[q]=='K':
                count['5']+=1
            if protein_seq[q]=='D' or protein_seq[q]=='E':
                count['6']+=1
            if protein_seq[q]=='C':
                count['7']+=1
        
        value=list(count.values())
        key=list(count.keys())
        maximum_occurence=0
        index=0
        for t in range(0, len(value)):
            if maximum_occurence < value[t]:
                maximum_occurence = value[t]
                index=t
        maximum_occurence = key[index] # group number of maximum occuring group
        for j in range(0, len(protein_seq)-k+1):
            kmer=protein_seq[j:j+k]
            c=''
            for l in range(0, k):
                if kmer[l] not in groups:
                    c += maximum_occurence
                else:
                    c+=groups[kmer[l]]
            V[combinations[int(c)]] += 1
            
        V = V/(len(protein_seq)-1)
        return np.array(V)
    
def get_protein_features(protein_sequence):
    aa=['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y']
    f = []
    protein_info = ProteinAnalysis(str(protein_sequence))
    protein_info.molecular_weight()
    amino_acid_percent = protein_info.get_amino_acids_percent()
    dp = []
    for a in aa:
        dp.append(amino_acid_percent[a])
    dp=np.array(dp)
    dp=normalize(np.atleast_2d(dp), norm='l2', copy=True, axis=1, return_norm=False)
    f.extend(dp[0])
    twomer=np.array(sequence_to_kmer(str(protein_sequence), 2))
    twomer=normalize(np.atleast_2d(twomer), norm='l2', copy=True, axis=1,return_norm=False)
    f.extend(twomer[0])
    threemer=np.array(sequence_to_kmer(str(protein_sequence), 3))
    threemer=normalize(np.atleast_2d(threemer), norm='l2', copy=True, axis=1,return_norm=False)
    f.extend(threemer[0])
    return np.array(f)

In [2]:
df = pd.read_csv("filtered_protacs.csv")
df.head()

,Target Protein,E3 Target,Warhead,Linker,E3 Ligand,Labels
0,MGKVNVAKLRYMSRDDFRVLTAVEMGMKNHEIVPGSLIASIASLKH...,MAGEGDQQDAAHNMGNHLPLLPAESEEEDEMEVEDQDSKEAKKPNI...,ON1CCC(CC1)NC(=O)C2=C(C=NN2)NC(=O)C3=C(C=CC=C3...,CCCCCCCC,NC1=CC=CC2=C1C(=O)N(C2=O)C3CCC(=O)NC3=O,1
1,MAVPFVEDWDLVQTLGEGAYGEVQLAVNRVTEEAVAVKIVDMKRAV...,MPRRAENWDEAEVGAEEAGVEEYGPEEDGGEESGAEESGPEESGPE...,OC1=CC2=C(N1)C(=O)N(C=C2C3=C(C=CC(=C3)NS(=O)(=...,COCCOCCOCCN,C(=O)N[C@H](C(=O)N1C[C@@H](C[C@H]1C(=O)NCC2=CC...,-1
2,MSGVSEPLSRVKLGTLRRPEGPAEPMVVVPVDVEKEDVRILKVCFY...,MAGEGDQQDAAHNMGNHLPLLPAESEEEDEMEVEDQDSKEAKKPNI...,ON1CCN(CC1)C2=CC=C(C=C2)NC3=NC=C4C(=N3)C(=CS4)...,CCOCCOCCO,C1=C2C(=CC=C1)C(=O)N(C2=O)C3CCC(=O)NC3=O,1
3,MSSLGASFVQIKFDDLQFFENCGGGSFGSVYRAKWISQDKEVAVKK...,MAGEGDQQDAAHNMGNHLPLLPAESEEEDEMEVEDQDSKEAKKPNI...,OC(=O)NC1=CC=CC=C1N,CCCCC,NC1=CC=CC2=C1C(=O)N(C2=O)C3CCC(=O)NC3=O,-1
4,MPNYKLTYFNMRGRAEIIRYIFAYLDIQYEDHRIEQADWPEIKSTL...,MPRRAENWDEAEVGAEEAGVEEYGPEEDGGEESGAEESGPEESGPE...,ON1CCN(CC1)CCOC2=CC3=C(C=C2)N4C=C(N=C4S3)C5=CC...,CCCCCCCCCC,C(=O)N[C@H](C(=O)N1C[C@@H](C[C@H]1C(=O)NCC2=CC...,-1


In [3]:
target_proteins = np.array(df["Target Protein"])
e3_targets = np.array(df["E3 Target"])
warheads = np.array(df["Warhead"])
linkers = np.array(df["Linker"])
e3_ligands = np.array(df["E3 Ligand"])
labels = np.array(df["Labels"])

In [4]:
temp = np.zeros(412)
target_protein_features = []
for p in tqdm(target_proteins):
    tp = get_protein_features(p)
    target_protein_features.append(tp)
    
e3_target_features = []
for p in tqdm(e3_targets):
    tp = get_protein_features(p)
    e3_target_features.append(tp)
    
e3_target_features = np.array(e3_target_features)
target_protein_features = np.array(e3_target_features)

print(e3_target_features.shape)
print(target_protein_features.shape)

100%|███████████████████████████████████████████████████████████████████████████| 10758/10758 [00:24<00:00, 443.16it/s]

(10758, 412)
(10758, 412)


In [5]:
warhead_fingerprints = []
for w in tqdm(warheads):
    temp = get_fingerprint(w)
    warhead_fingerprints.append(temp)
    
no_linker_count = 0
linker_fingerprints = []
for l in tqdm(linkers):
    if type(l) == float:
        temp_array = np.zeros(1024)
        linker_fingerprints.append(temp_array)
        no_linker_count += 1
        continue
    temp = get_fingerprint(l)
    linker_fingerprints.append(temp)
    
e3_ligand_fingerprints = []
midway_bindings = 0
for e in tqdm(e3_ligands):
    if "[R2]" in e:
        e = e.replace("[R2]", "O")
        midway_bindings += 1
    temp = get_fingerprint(e)
    e3_ligand_fingerprints.append(temp)
    
warhead_fingerprints = np.array(warhead_fingerprints)
linker_fingerprints = np.array(linker_fingerprints)
e3_ligand_fingerprints = np.array(e3_ligand_fingerprints)
print("number of protacs with no linker: ", no_linker_count)
print("number of e3 ligands with midway bindings: ", midway_bindings)

100%|██████████████████████████████████████████████████████████████████████████| 10758/10758 [00:04<00:00, 2627.60it/s]

number of protacs with no linker:  30
number of e3 ligands with midway bindings:  12


In [10]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# Combine the features together
combined_features = np.column_stack((target_protein_features, e3_target_features,
                                     warhead_fingerprints, linker_fingerprints,
                                     e3_ligand_fingerprints))

data = []
for i in range(len(combined_features)):
    data.append([combined_features[i], labels[i]])
        
print(combined_features.shape)
train_size = 8000
val_size = (int) ((len(combined_features)-train_size) / 2)
train_dataset = data[:train_size]
val_dataset = data[train_size:train_size+val_size]
test_dataset = data[train_size+val_size:]


# Feed forward Neural network Implementation
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torch.autograd import Variable
from sklearn.metrics import roc_auc_score, accuracy_score, average_precision_score
USE_CUDA = torch.cuda.is_available() 
from tqdm import tqdm as tqdm

def cuda(v):
    if USE_CUDA:
        return v.cuda()
    return v

def toTensor(v, dtype=torch.float, requires_grad=False):
    return cuda(Variable(v.clone().detach()).type(dtype).requires_grad_(requires_grad))

def toNumpy(v):
    if USE_CUDA:
        return v.detach().cpu().numpy()
    return v.detach().numpy()

# Hyper Parameters
no_of_features = 412*2 + 1024*3
hidden_size = 500
output_size = 1
num_epochs = 20
batch_size = 100
learning_rate = 0.001

# Load datasets
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

val_loader = torch.utils.data.DataLoader(dataset=val_dataset,
                                         batch_size=batch_size,
                                         shuffle=False)


# Neural Network with 2 hidden layers
class Net(nn.Module):
    def __init__(self, no_of_features, hidden_size, output_size):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(no_of_features, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        out = self.fc3(out)
        return out
        
model = cuda(Net(no_of_features, hidden_size, output_size))

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    
model.train() 

for i, epoch in enumerate(range(num_epochs)):
    losses = []
    for examples, my_labels in train_loader:
        examples = toTensor(examples, dtype=torch.float32)
        my_labels = toTensor(my_labels, dtype=torch.float32)
        my_labels = my_labels.unsqueeze(1)
        optimizer.zero_grad()
        output = model(examples)
        loss = criterion(output, my_labels)
        my_loss = (float) (loss)
        losses.append(my_loss)
        loss.backward()
        optimizer.step()
    losses = np.array(losses)
    print("Epoch ", i, " loss: ", losses.sum())
        
# Validation
model.eval()
correct = 0
total = 0
threshold = 0
for examples, my_labels in tqdm(val_loader):
    examples = toTensor(examples, dtype=torch.float32)
    output = model(examples)
    predictions = []
    for o in output:
        o = (float) (o)
        if threshold > 0:
            predictions.append(1.0)
        else:
            predictions.append(-1.0)
    predictions = np.array(predictions)
    my_labels = toNumpy(my_labels)
    total += len(my_labels)
    correct += (predictions == my_labels).sum()
    print(total, " ", correct)
    
print('Accuracy of the network on the protac dataset: %d %%' % (100 * correct / total))

(10758, 3896)
Epoch  0  loss:  61.75151699781418
Epoch  1  loss:  35.80623349547386
Epoch  2  loss:  22.594202145934105
Epoch  3  loss:  15.222537644207478
Epoch  4  loss:  10.10051092505455
Epoch  5  loss:  7.559809589758515
Epoch  6  loss:  5.591318758204579
Epoch  7  loss:  4.769012155011296
Epoch  8  loss:  3.963257341645658
Epoch  9  loss:  3.6344493981450796
Epoch  10  loss:  3.068289832212031
Epoch  11  loss:  3.3326302361674607
Epoch  12  loss:  3.159964757040143
Epoch  13  loss:  2.7190077332779765
Epoch  14  loss:  2.756417401600629
Epoch  15  loss:  2.819523736834526
Epoch  16  loss:  3.0212050625123084
Epoch  17  loss:  2.5502129890955985
Epoch  18  loss:  2.643561640754342
Epoch  19  loss:  2.3957861228846014


100%|█████████████████████████████████████████████████████████████████████████████████| 14/14 [00:00<00:00, 189.69it/s]

100   44
200   100
300   148
400   193
500   246
600   293
700   344
800   399
900   444
1000   500
1100   555
1200   598
1300   655
1379   696
Accuracy of the network on the protac dataset: 50 %
